In [1]:
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": ""
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": ""
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": ""
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": ""
    }
]

In [2]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [3]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [4]:
%%time
session = get_session("EMPLOYEE_10L","0e0fb803-22db-4d88-9f2f-f6f75b6abcf0")

CPU times: user 94.1 ms, sys: 2.15 ms, total: 96.2 ms
Wall time: 577 ms


In [5]:
%%time
reg = Registry(session=session)

CPU times: user 67 µs, sys: 30 µs, total: 97 µs
Wall time: 100 µs


In [6]:
%%time
model_list = session.sql("show models like '%exp6789%'").collect()

CPU times: user 36.5 ms, sys: 47.9 ms, total: 84.4 ms
Wall time: 2.39 s


In [ ]:
model_list

In [7]:
%%time
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    for run in monitor_resp:
        if run.get('run_id') == model_run_id:
            run.update(metrics)
print(monitor_resp)

[{'run_id': '8042932c-101a-4ddc-a808-00c092797960', 'algorithm_used': 'XGBClassifier', 'status': 'Successful', 'model_metrics': {'accuracy_score': 0.912834, 'f1_score': 0.863711823763472, 'recall_score': 0.8009236633760141, 'precision_score': 0.9371819065591582, 'roc_auc': 0.8863324874130605, 'log_loss': 3.1437201566897612}}, {'run_id': '16e3f179-bb70-4632-9a39-9a6fac4a5581', 'algorithm_used': 'DecisionTreeClassifier', 'status': 'Successful', 'model_metrics': {'accuracy_score': 0.92473, 'f1_score': 0.8841251148525358, 'recall_score': 0.8361769313619897, 'precision_score': 0.9379067021773153, 'roc_auc': 0.9036114375207266, 'log_loss': 2.7149916732535027}}, {'run_id': 'ef47a0a5-1a74-4a14-b0fd-8dbbd322057f', 'algorithm_used': 'GradientBoostingClassifier', 'status': 'Running'}, {'run_id': '26a15f9c-29fd-4d55-82fb-3d38cc18e4b8', 'algorithm_used': 'RandomForestClassifier', 'status': 'Running'}]
CPU times: user 401 ms, sys: 72.6 ms, total: 473 ms
Wall time: 4.1 s


In [ ]:
%%time
m = reg.get_model("MY_RUN_LISTING_EXP_TEST12345_EXP6789_16E3F179_BB70_4632_9A39_9A6FAC4A5581")

In [ ]:
dir(m)

In [ ]:
m.show_tags()

In [ ]:
m.default.get_metric('model_metrics')

In [ ]:
####Approach 2####
